## NY Motor Vehicle Collisions —— Exploratory Data Analysis
Source：New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95

The dataset I have at hand comprises information extracted from police reports, specifically the MV104-AN, which is mandatory for documenting motor vehicle collisions in New York City (NYC) involving injuries, fatalities, or property damage exceeding $1000.

The dataset covers incidents from July 1, 2012, onward. However, for the purpose of this analysis, I will focus on the period up to August 15, 2023, which is when I obtained the data.

I have access to more than 2 million records within this dataset, which presents an opportunity to investigate various facets of vehicular accidents in the different boroughs of NYC.

Commencing with an exploratory data analysis, I aim to address the following questions:

- I intend to identify the borough with the highest reported crash count since 2012.

- I will correlate the crash counts with each borough's population to determine which borough has the highest number of crashes per 100,000 people. Although the dataset lacks population information, merging the crash data with population estimates available from the City of New York's website will enable this analysis:
(https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 | 

- The primary cause of crashes will be investigated.

- I will identify the top three causes of crashes and calculate what proportion of all crashes are caused by these top-3 causes.

- To gauge the severity of accidents, I will determine how many involved at least one fatality.

- Calculating the average number of fatal accidents per 1000 accidents is on my agenda.

- Checking for missing data and calculating the proportion of accidents in the dataset without a borough code is also a part of this analysis.

- I will scrutinize the 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' fields to identify the combinations of vehicles involved in the highest number of accidents.



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [2]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df


,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


#### 1.The borough with the highest reported crash count since 2012

In [3]:
# Filter out rows with missing values in the 'BOROUGH' column
crash_data_filtered = df.dropna(subset=['BOROUGH'])
# Group by 'BOROUGH' and count the number of crashes in each borough
borough_crash_counts = crash_data_filtered['BOROUGH'].value_counts()
# Find the borough with the highest crash count
highest_crash_borough = borough_crash_counts.idxmax()
highest_crash_count = borough_crash_counts.max()
f"The borough with the highest number of crashes since 2012 is {highest_crash_borough} with {highest_crash_count} crashes."

'The borough with the highest number of crashes since 2012 is BROOKLYN with 441026 crashes.'

#### 2.The Borough with the maximum number of crashes adjusted for population

In [4]:
# Population data
population_data = {
    'Borough': ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
    'Population': [1446788, 2648452, 1638281, 2330295, 487155]
}
population_df = pd.DataFrame(population_data)

In [5]:
crash_data_filtered = df.dropna(subset=['BOROUGH'])
# Group by 'BOROUGH' and count the number of crashes in each borough
borough_crash_counts = crash_data_filtered['BOROUGH'].value_counts()

In [6]:
population_df.set_index('Borough')
population_df.index = [str(x).upper() for x in population_df['Borough']]

In [7]:
# Calculate crash rate per 100,000 people for each borough
borough_crash_rates = (borough_crash_counts / population_df['Population']) * 100000

In [8]:
highest_crash_rate_borough = borough_crash_rates.idxmax()
highest_crash_rate = borough_crash_rates.max()
round(highest_crash_rate)

19122

#### 3.The leading cause of crashes

In [9]:
# Filter out rows where 'CONTRIBUTING FACTOR VEHICLE 1' is 'Unspecified'
crash_data_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
# Count the occurrences of each contributing factor
contributing_factor_counts = crash_data_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()
# Find the leading cause of crashes
leading_cause = contributing_factor_counts.index[0]
leading_cause_count = contributing_factor_counts.iloc[0]
# The proportion of accidents are attributable to this cause
total_crashes = len(crash_data_filtered)
proportion_of_accidents = (leading_cause_count / total_crashes) * 100

print(leading_cause)
print(leading_cause_count)
print(str(round(proportion_of_accidents, 1)) + '%')

Driver Inattention/Distraction
401262
30.3%


#### 4.What proportion of total crashes do the top 3 causes account for (excluding 'Unspecified')

In [29]:
# Count the occurrences of each contributing factor
contributing_factor_counts = crash_data_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()
# Get the top 3 causes
top_3_causes = contributing_factor_counts.head(3)
# Calculate the proportion of total crashes accounted for by the top 3 causes
total_crashes = len(crash_data_filtered)
proportion_of_top_3_causes = (top_3_causes.sum() / len(crash_data_filtered)) * 100

print(top_3_causes.index.values)
print(str(round(proportion_of_top_3_causes, 1)) + '%')

['Driver Inattention/Distraction' 'Failure to Yield Right-of-Way'
 'Following Too Closely']
47.4%


#### 5.Analyzing fatalities

##### What is the cumulative count of accidents involving at least one fatality when considering the complete data since 2012 (accounting for any missing values if applicable).

In [10]:
# Filter the dataset to include only rows where 'NUMBER OF PERSONS KILLED' is greater than 0
fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] > 0]
# Calculate the total count of accidents with at least one fatality
total_fatal_accidents = len(fatal_accidents)
total_fatal_accidents

2804

#### 6.Fatal accidents

##### On average, for every 1000 accidents, how many have resulted in at least one person dead?

In [15]:
# Calculate the total number of accidents in the dataset
total_accidents = len(df)
# Calculate the average number of fatal accidents per 1000 accidents
average_fatalities_per_1000 = (total_fatal_accidents / total_accidents) * 1000
average_fatalities_per_1000

1.3893258747079764

#### 7.The proportion of accidents in the data do not have a Borough code

In [16]:
# Count the number of rows where 'BOROUGH' is missing (NaN)
missing_borough_count = df['BOROUGH'].isna().sum()
# Calculate the proportion of accidents without a Borough code
proportion_missing_borough = (missing_borough_count / total_accidents) * 100
print(str(round(proportion_missing_borough, 1)) + '%')

31.1%


#### 8.Which combination of vehicles have the most number of accidents

In [17]:
# Create a new DataFrame with only the relevant columns
vehicle_combinations = df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']]
# Count the occurrences of each combination of vehicle types
vehicle_combination_counts = vehicle_combinations.value_counts()
# Get the combination with the highest count
most_common_combination = vehicle_combination_counts.idxmax()
most_common_count = vehicle_combination_counts.max()

print(most_common_combination)
print(most_common_count)

('Sedan', 'Sedan')
197944
